# 1. Data Retrieval

**TODO:** Copy from 

One for manual inspection (csv) and one conservative (pkl)

Loads necessary libraries for the EDA:

In [ ]:
import os
import subprocess
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv

Global configs:

In [ ]:
# Path to root directory of the repo.
root_dir_ = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"],
    text=True,
)
ROOT_DIR = root_dir_.strip()

# Path to data directory.
DATA_DIR = os.path.join(ROOT_DIR, "data")

# Paths to which dataframes will be saved
DF_PKL_PATH = os.path.join(DATA_DIR, "df.pkl")
DF_CSV_PATH = os.path.join(DATA_DIR, "df.csv") 

After exploring in db beaver. We jouin on id patient id

In [ ]:
QUERY = """
    SET SCHEMA 'diabetes';
    SELECT * FROM
            patient p
        LEFT JOIN
            blood_metrics bm
        ON p.id = bm.patientid
        LEFT JOIN
            pedigree_outcome po
        ON p.id = po.patientid
        LEFT JOIN
            skin s
        ON p.id = s.patientid
    ;
"""

Loads data from the database and stores it in pandas dataframes:

In [ ]:
load_dotenv()

DB_CONFIG = {
    "database": os.getenv("DATABASE"),
    "scheme": os.getenv("SCHEME"),
    "user": os.getenv("USER"),
    "password": os.getenv("PASSWORD"),
    "host": os.getenv("HOST"),
    "port": os.getenv("PORT")
}

DB_STRING = (
    "{scheme}://{user}:{password}@{host}:{port}/{database}"
    .format(**DB_CONFIG)
)

print(DB_STRING)

In [ ]:
db = create_engine(DB_STRING)

with db.connect() as conn:
    df = pd.read_sql(QUERY, conn)

Let us check that the data were retrieved.

In [ ]:
df.info()

In [ ]:
df.sample(10)



Before storing, let us remove 

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]

In [ ]:
df.to_pickle(DF_PKL_PATH)
df.to_csv(DF_CSV_PATH, index=False)